In [1]:
import pandas as pd
import re

In [2]:
df = pd.read_excel('../acts_and_miracles/acts_and_miracles_preprep.xlsm', sheet_name='acts_and_miracles')
#df = pd.read_excel('../acts_and_miracles/acts_and_miracles.xlsx', sheet_name='acts_and_miracles')
df = df.dropna(how='all')
df

Description         Category  \
0                     Creation of the heavens and earth         Creation   
1                       God speaks light into existence  Speech/Creation   
2     God speaks to separate waters and create firma...  Speech/Creation   
3           God speaks dry land and seas into existence  Speech/Creation   
4                  God speaks vegetation into existence  Speech/Creation   
...                                                 ...              ...   
2977           Tree of life bears twelve kinds of fruit        Provision   
2978            No more curse, night, or need for light   Transformation   
2979                      Angel shows John these things       Revelation   
2980                  Jesus declares His coming quickly          Promise   
2981                        Spirit and Bride say "Come"       Invitation   

                    Reference          Location               Date  \
0             Genesis 1:1-2:3             Earth  Beginning of Time   
1                 Genesis 1:3             Earth  Beginning of Time   
2               Genesis 1:6-8             Earth  Beginning of Time   
3              Genesis 1:9-10             Earth  Beginning of Time   
4             Genesis 1:11-13             Earth  Beginning of Time   
...                       ...               ...                ...   
2977          Revelation 22:2     New Jerusalem             Future   
2978        Revelation 22:3-5     New Jerusalem             Future   
2979        Revelation 22:6-9  Island of Patmos           c. 95 AD   
2980  Revelation 22:7, 12, 20  Island of Patmos           c. 95 AD   
2981         Revelation 22:17  Island of Patmos           c. 95 AD   

               Messenger       Supporting References/Notes  
0                    GOD                               NaN  
1                    GOD                               NaN  
2                    GOD                               NaN  
3                    GOD                               NaN  
4                    GOD                               NaN  
...                  ...                               ...  
2977                 God  Genesis 2:9; 3:22; Ezekiel 47:12  
2978                 God      Genesis 3:17; Zechariah 14:7  
2979               Angel                 Daniel 8:26; 12:4  
2980        Jesus Christ        Malachi 3:1; Matthew 16:27  
2981  Holy Spirit/Church                       Isaiah 55:1  

[2982 rows x 7 columns]

In [3]:
df.columns

Index(['Description', 'Category', 'Reference', 'Location', 'Date', 'Messenger',
       'Supporting References/Notes'],
      dtype='object')

In [9]:
def extract_words_left_of_numbers(text):
    """
    Extract all words left of middle-occurring numbers,
    or leftmost number+words if starts with number
    """
    if pd.isna(text):
        return ""
    
    text = str(text).strip()
    
    # Case 1: Starts with number - get number + following words until next number
    if re.match(r'^\d', text):
        pattern = r'^(\d+\s*[A-Za-z]+(?:\s+[A-Za-z]+)*)'
        match = re.search(pattern, text)
        return match.group(1).strip() if match else ""
    
    # Case 2: Starts with letters - get all words before first number
    else:
        pattern = r'^([A-Za-z]+(?:\s+[A-Za-z]+)*)'
        match = re.search(pattern, text)
        return match.group(1).strip() if match else ""

num_df = pd.read_excel('../acts_and_miracles/acts_and_miracles_preprep.xlsx', sheet_name='book_to_number', header=0)
num_df

ref_df = pd.read_excel('../acts_and_miracles/acts_and_miracles_preprep.xlsx', sheet_name='acts_and_miracles')
ref_df['Reference'] = ref_df['Reference'].str.replace('Psalm', 'Psalms')
ref_df

ref_df['book_name'] = [extract_words_left_of_numbers(ref) for ref in ref_df.Reference]
ref_df

mergeddf = pd.merge(ref_df, num_df, how='left', on='book_name')
mergeddf
mergeddf.to_excel('../acts_and_miracles/acts_and_miracles.xlsx', sheet_name='acts_and_miracles', index=False)

In [ ]:
def extract_year_and_era(df, column_name, year_col='first_year', era_col='era'):
    """
    Extract first year and BC/AD era into separate columns
    
    Parameters:
    - df: DataFrame
    - column_name: name of column containing date strings
    - year_col: name for new year column
    - era_col: name for new era column
    """
    
    def parse_date_string(text):
        """
        Parse date string and return (year, era)
        """
        if pd.isna(text) or text == '':
            return None, None
        
        text = str(text).strip().upper()
        
        # Remove common prefixes
        prefixes = r'^(CIRCA|C\.|ABOUT|CA\.|AROUND|APPROXIMATELY|APPROX\.?)\s*'
        text = re.sub(prefixes, '', text, flags=re.IGNORECASE)
        
        # Pattern to capture year and era
        pattern = r'(\d{1,4})(?:\s*-\s*\d{1,4})?\s*(BC|AD|BCE|CE|BC\.?|AD\.?)?'
        
        matches = re.findall(pattern, text)
        
        if matches:
            year_str, era = matches[0]
            year = int(year_str)
            
            # Standardize era values
            if era:
                era = era.replace('.', '').upper()
                if era in ['BC', 'BCE']:
                    era = 'BC'
                elif era in ['AD', 'CE']:
                    era = 'AD'
            else:
                # If no era specified and year > 1000, assume AD
                era = 'AD' if year > 1000 else None
            
            return year, era
        
        return None, None
    
    # Apply parsing function
    parsed_data = df[column_name].apply(parse_date_string)
    
    # Split into separate columns
    df[year_col] = [item[0] for item in parsed_data]
    df[era_col] = [item[1] for item in parsed_data]
    
    return df

In [ ]:
result_df = extract_year_and_era(df, 'Date')
#df.to_excel('../acts_and_miracles/acts_and_miracles.xlsx', sheet_name='acts_and_miracles', index=False)

In [ ]:
def alphabetize_slash_strings_replace_nan(string_list):
    return ['/'.join(sorted(str(s).split('/'))) if pd.notna(s) else '' for s in string_list]

result = alphabetize_slash_strings_replace_nan(df.Category.tolist())

df.Category = result
df.Category

df.head(20)

In [8]:
#df.to_excel('../acts_and_miracles/acts_and_miracles.xlsx', sheet_name='acts_and_miracles', index=False)